Forward Discrete Fourier Transform (DFT):
$$
    F(u) = \sum_{x=0}^{M-1} f(x) e^{-j2\pi ux / M}, 
$$
for $u =0,1,\ldots,M-1$.

Inverse Discrete Fourier Transform (I-DFT):
$$
    f(x) = \sum_{u=0}^{M-1} F(u) e^{j2\pi ux / M}, 
$$
for $x =0,1,\ldots,M-1$.

$f(x) \rightarrow F(u)$ is equivalent to this matrix-vector $F = (F(0), \ldots, F(M-1))^T = A \times (f(0), \ldots, f(M-1))^T$.

In [1]:
import numpy as np

In [2]:
def DFT(f):
    f = np.asarray(f, dtype=np.float)
    M = f.shape[0]
    x = np.arange(M)
    u = np.arange(M)
    A = np.exp(-1j * 2 * np.pi * np.vstack([u for i in range(M)]).T * x / M)
    F = A @ f
    return F

In [3]:
f = np.arange(10)
f

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [4]:
F_dft = DFT(f)
F_dft

array([45.+0.00000000e+00j, -5.+1.53884177e+01j, -5.+6.88190960e+00j,
       -5.+3.63271264e+00j, -5.+1.62459848e+00j, -5.+3.53452967e-14j,
       -5.-1.62459848e+00j, -5.-3.63271264e+00j, -5.-6.88190960e+00j,
       -5.-1.53884177e+01j])

In [5]:
F_fft = np.fft.fft(f)
F_fft

array([45. +0.j        , -5.+15.38841769j, -5. +6.8819096j ,
       -5. +3.63271264j, -5. +1.62459848j, -5. +0.j        ,
       -5. -1.62459848j, -5. -3.63271264j, -5. -6.8819096j ,
       -5.-15.38841769j])

In [6]:
np.allclose(F_fft, F_dft)

True

In [7]:
f = np.arange(1024)

In [8]:
%timeit DFT(f)

98.1 ms ± 8.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit np.fft.fft(f)

8.25 µs ± 91.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Fast Fourier Transform

Computing $F(u)=\sum_{x=0}^{M-1} f(x) e^{-j2\pi u x / M}$ is equivalent to matrix-vector $F = A \times f$.

$F(u+M) = \sum_{x=0}^{M-1} f(x) e^{\frac{-j2\pi (u+M) x}{M}}$

$ = \sum_{x=0}^{M-1} f(x) e^{\frac{-j2\pi u x}{M}} \times e^{\frac{-j2\pi M x}{M}}$

$ = \sum_{x=0}^{M-1} f(x) e^{\frac{-j2\pi u x}{M}}$

$ = F(u)$

Similarly, we can show $F(u+iM) = F(u)$ for $i\in\mathbb{N} \cup \{0\}$. We call this __symmetric property__.

$ F(u) = \sum_{x=0}^{M-1} f(x) e^{\frac{-j2\pi u x}{M}}$

$ = \sum_{x=0}^{M/2-1} f(2x) e^{\frac{-j2\pi u (2x)}{M}} + $

$ \sum_{x=0}^{M/2-1} f(2x+1) e^{\frac{-j2\pi u (2x + 1)}{M}}$

$ = \sum_{x=0}^{M/2-1} f(2x) e^{\frac{-j2\pi u x}{M/2}} + $

$ e^{\frac{-j2\pi u}{M}}\sum_{x=0}^{M/2-1} f(2x+1) e^{\frac{-j2\pi u x}{M/2}}$ 

In [10]:
def fft(f):
    f = np.asarray(f, dtype=np.float)
    M = f.shape[0]
    if M % 2 > 0 or M < 32:
        F = DFT(f)
    else:
        # using the divide & conquer approach to compute the DFT
        F_even = fft(f[::2])
        F_odd = fft(f[1::2])
        
        coeff = np.exp(-1j * 2 * np.pi * np.arange(M) / M)
        F = np.concatenate((F_even + coeff[:M//2] * F_odd, F_even + coeff[M//2:] * F_odd))
    return F

In [11]:
f = np.arange(1024)

In [12]:
%timeit F_fft = np.fft.fft(f)

8.37 µs ± 82 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [13]:
%timeit F_dft = DFT(f)

97.1 ms ± 3.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%timeit F_myfft = fft(f)

7.89 ms ± 55 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
